# Personal Finance Tracker — Bank Statements Cleaning

Developed automated cleaning script for bank statements to simplify monthly budgeting. This notebook runs the cleaning pipeline, displays samples of the cleaned data, and shows monthly summaries and category breakdowns.

## Run the automated cleaning script

The next cell executes the robust cleaning script located in ../scripts/cleanup.py. The script produces cleaned CSV files in ../data/cleaned/.

In [ ]:
import subprocess, sys, os

# Run the cleaning script (prints a short summary and creates cleaned files)
script_path = os.path.join('..', 'scripts', 'cleanup.py')
print('Executing:', script_path)
subprocess.run([sys.executable, script_path], check=True)

## Load cleaned outputs

After the script runs, load the cleaned dataset and monthly summary for inspection.

In [ ]:
import pandas as pd, os

base = os.path.normpath(os.path.join('..'))
clean_dir = os.path.join(base, 'data', 'cleaned')
cleaned_csv = os.path.join(clean_dir, 'cleaned_bank_statements.csv')
monthly_csv = os.path.join(clean_dir, 'monthly_summary.csv')

df = pd.read_csv(cleaned_csv, parse_dates=['Date'])
monthly = pd.read_csv(monthly_csv)

print('Cleaned rows:', len(df))
df.head(10)

## Data snapshot and quick checks

Inspect a few cleaned rows and ensure Date, Description, Amount, Category, Balance and Anomaly columns are present.

In [ ]:
# Show cleaned sample with anomalies highlighted
display_cols = ['Date','Description','Amount','Category','Balance','Anomaly']
df[display_cols].head(15)

## Monthly summary

The script also generates per-month totals: income, expenses, net and transaction count.

In [ ]:
monthly.head(20)

## Next steps (suggestions)

- Use the cleaned CSV in a budgeting/dashboard tool (e.g., Power BI, Excel, or Plotly Dash).
- Extend category synonyms to capture more merchant-specific patterns.
- Integrate with personal income/expense forecasts.